# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [1]:
import numpy as np
from symred.utils import unit_n_sphere_cartesian_coords
from symred.symplectic_form import PauliwordOp, QuantumState
from functools import cached_property
from typing import Union

Firstly, note the correct phases under multiplication by a Pauli Y:

In [2]:
zero = QuantumState([[0]])
one = QuantumState([[1]])
Y = PauliwordOp(['Y'], [1])

print(f'{zero} -> {Y * zero}')
print(f'{one} -> {Y * one}')

 1.0000000000 |0> ->  0.0000000000+1.0000000000j |1>
 1.0000000000 |1> ->  0.0000000000-1.0000000000j |0>


Now, let's see what happens when we apply the Hadamard gate to the zero state:

In [3]:
psi = QuantumState([[0,0]])
had = PauliwordOp(['XZ','ZX','XX', 'ZZ'], np.ones(4)/2) # 2-qubit Hadamard gate decomposed over Paulis
eq_superposition = had * psi
print(f'Zero state: {psi}\n')
print(f'After application of the Hadamard gate:\n\n{eq_superposition}')

Zero state:  1.0000000000 |00>

After application of the Hadamard gate:

 0.5000000000+0.0000000000j |00> +
 0.5000000000+0.0000000000j |01> +
 0.5000000000+0.0000000000j |10> +
 0.5000000000-0.0000000000j |11>


Observe that the QuantumState is represented internally by its `state_op`, a PauliwordOp object that governs its behaviour under multiplication

In [4]:
print(eq_superposition.state_op)

0.5000000000+0.0000000000j ZZ +
0.5000000000+0.0000000000j ZX +
0.5000000000+0.0000000000j XZ +
0.5000000000-0.0000000000j XX


Try evaluating expectation values for randomly generated state and Hermitian operators:

In [5]:
def random_state(num_qubits, num_terms):
    """ Generates a random normalized QuantumState
    """
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(num_terms,num_qubits))
    # random vector of coefficients
    coeff_vec = (
        np.random.rand(num_terms) + 
        np.random.rand(num_terms)*1j
    )
    return QuantumState(random_state, coeff_vec).normalize

def random_herm_op(num_qubits, num_terms):
    """ Generates a random PaulwordOp
    """
    # random binary array with 2N columns, M rows
    random_symp_mat = np.random.randint(0,2,(num_terms,num_qubits*2))
    # random vector of coefficients
    coeff_vec = np.random.rand(num_terms)
    coeff_vec/=coeff_vec[::-1]
    coeff_vec*=(2*np.random.randint(0,2,num_terms)-1)
    return PauliwordOp(random_symp_mat, coeff_vec)

In [6]:
N = 4 # number of qubits
M = 2 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)

print(psi_1)
print()
print(psi_2)

 0.3985173978+0.7233753686j |1011> +
 0.5537726046+0.1060559391j |0101>

 0.6993170211+0.2655889463j |0111> +
 0.6538480047+0.1135825798j |1001>


In [7]:
psi = (psi_1 + psi_2).normalize
print(psi)
print()
print('Norm:', psi.conjugate * psi)

 0.3915763639+0.0749928737j |0101> +
 0.4944918078+0.1877997450j |0111> +
 0.4623403580+0.0803150124j |1001> +
 0.2817943544+0.5115036285j |1011>

Norm: (1+0j)


In [8]:
op = random_herm_op(N, 15)
print(op)
print()
print('Expectation value <psi|op|psi> =', psi.conjugate * op * psi)

1.6140737073+0.0000000000j YIZY +
0.7095530362+0.0000000000j ZIXI +
-2.3766391112+0.0000000000j YYYX +
0.4689553394+0.0000000000j YIXX +
-0.6282655340+0.0000000000j IYIX +
0.2433565895+0.0000000000j YYIZ +
0.2278358649+0.0000000000j ZIIX +
-1.0000000000+0.0000000000j IIYI +
4.3891246033+0.0000000000j YZXX +
4.1091963122+0.0000000000j ZZXX +
-1.5916836845+0.0000000000j XIYI +
2.1323992203+0.0000000000j XXXZ +
0.4207622416+0.0000000000j IXZY +
-1.4093379197+0.0000000000j IIXI +
-0.6195503932+0.0000000000j ZZXI

Expectation value <psi|op|psi> = (-2.926997156382183-1.1102230246251565e-16j)


In [9]:
op_psi = op * psi
print(op_psi)

 1.7231128342-1.4173604660j |0000> +
-0.8141519800+0.4485274763j |0001> +
 0.6480064602-0.9017462206j |0010> +
 0.1278360948-0.7358996045j |0011> +
-3.1584083024-0.0848964693j |0100> +
-0.9408866774-0.6308513350j |0101> +
-1.3055213823-1.3641891003j |0110> +
-1.0108957881-0.6933348272j |0111> +
-1.7096182561-0.9449388935j |1000> +
-2.0837559681-0.9038378235j |1001> +
-1.7858191576-1.3772047258j |1010> +
-1.5682256131-0.7883769647j |1011> +
 1.4524925802-2.3024554377j |1100> +
-0.2989177900+0.7870745426j |1101> +
-0.3980227799-1.8966415133j |1110> +
 0.1193649336-0.6232657097j |1111>


In [11]:
psi.conjugate * op_psi

(-2.926997156382183+1.1102230246251565e-16j)